### Import 

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from spacy.lang.fr import stop_words
from sklearn.linear_model import LogisticRegression
from mlflow.models import infer_signature
stop_words = list(stop_words.STOP_WORDS)
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import mlflow
import subprocess
from mlflow import MlflowClient

### Connect to server and enable MlFlow.sklearn autolog

In [2]:
commit_hash = subprocess.check_output(["git", "rev-parse", "HEAD"])

In [3]:
remote_server_uri = "http://localhost:5000/"
mlflow.set_tracking_uri(remote_server_uri)
mlflow.sklearn.autolog(log_datasets=False)
mlflow.set_experiment("/1st-experiment")
run = mlflow.start_run()
mlflow.set_tags({
"mlflow.note.content" : "This is MLOPS project",
"mlflow.source.name" : "model_design_2.ipynb",
"mlflow.source.git.commit" : commit_hash,
"mlflow.source.git.branch" : "HEAD",
})



### Loading data

In [4]:
df_train = pd.read_csv("../../data/train.csv")
df_test = pd.read_csv("../../data/test.csv")
df_valid = pd.read_csv("../../data/valid.csv")

### Manual logging for dataset

### Pipeline

In [5]:
pipe = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words=stop_words)),
    ("logreg", LogisticRegression( solver="liblinear"))
])

pipe.fit(df_train["review"], df_train["polarity"])
y_pred = pipe.predict(df_test["review"])

/home/ainazeaze/Etude/MLOPS/mlops_project/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(
2025/11/18 17:20:09 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/18 17:20:10 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


### Add test accuracy score

In [6]:
Y_test = df_test["polarity"].to_numpy()
test_accuracy_score = accuracy_score(pipe.predict(df_test["review"]), Y_test)
mlflow.log_metric("test_accuracy_score", test_accuracy_score)

### MlFlow Registry

In [ ]:
signature = infer_signature(df_test["review"], y_pred)
logged_model = mlflow.sklearn.log_model(pipe, name="reg_model", signature=signature, registered_model_name="reg_model")

# Add tag and description to a registered model
client = MlflowClient()
desc = "Polarity estimation from text model"
client.set_registered_model_tag("reg_model", "owner", "aina" )
client.update_registered_model("reg_model", description=desc)


2025/11/18 17:20:48 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'reg_model'.
2025/11/18 17:20:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: reg_model, version 1
Created version '1' of model 'reg_model'.


<RegisteredModel: aliases={}, creation_timestamp=1763482848466, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='Polarity estimation from text model', last_updated_timestamp=1763482848938, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1763482848521, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1763482848521, metrics=None, model_id=None, name='reg_model', params=None, run_id='083cb3d46ce44a3996a7617950453b64', run_link='', source='models:/m-739c07760b7b439088ce49687563f0f0', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='reg_model', tags={'owner': 'aina'}>

### Close mlflow run

In [8]:
mlflow.end_run()

🏃 View run dazzling-flea-467 at: http://localhost:5000/#/experiments/529200409768567991/runs/083cb3d46ce44a3996a7617950453b64
🧪 View experiment at: http://localhost:5000/#/experiments/529200409768567991
